### **Importing Libraries**

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.utils.data import Dataset
import torchvision.models as models
from torchvision import transforms
import torchvision
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchmetrics.segmentation import MeanIoU
import torchvision.transforms as T
from PIL import Image
import math
from sklearn.preprocessing import StandardScaler
from scipy import stats
from torchvision.models.vision_transformer import vit_b_16
import csv
import timm
from torch_ema import ExponentialMovingAverage
import csv

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### **Dataset**

#### **Helper Functions**

In [38]:
transform_base = T.Compose([
    T.Resize((224, 224)),
    T.RandAugment(num_ops=2, magnitude=9),
    T.ColorJitter(brightness=0.4, contrast=0.3, saturation=0.3, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ,
    T.RandomErasing(p=0.25, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
])

transform_color = T.Compose([
    T.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    T.RandomErasing(p=0.25, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
])

transform_affine = T.Compose([
    T.Resize((288, 288)),
    T.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.85, 1.15), shear=10),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    T.RandomErasing(p=0.25, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
])

transform_val = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def detect_outliers(df, columns, iqr_multiplier=1.35):
    mask = np.ones(len(df), dtype=bool)

    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR

        col_mask = (df[col] >= lower_bound) & (df[col] <= upper_bound)
        mask &= col_mask

    return mask 

def preprocess_coordinates(df, stats=None, ids_to_remove=None):
    df_clean = df.copy()
    
    if df_clean['latitude'].isnull().sum() > 0 or df_clean['longitude'].isnull().sum() > 0:
        print(f"Warning: Found {df_clean['latitude'].isnull().sum()} missing latitude values and "
              f"{df_clean['longitude'].isnull().sum()} missing longitude values")
        df_clean['latitude'] = df_clean['latitude'].fillna(df_clean['latitude'].median())
        df_clean['longitude'] = df_clean['longitude'].fillna(df_clean['longitude'].median())
    
    if stats is None:
        outlier_mask = detect_outliers(df_clean, ['latitude', 'longitude'])
        outliers_count = (~outlier_mask).sum()
        if outliers_count > 0:
            print(f"Removed {outliers_count} outliers from dataset")
            df_clean = df_clean[outlier_mask].reset_index(drop=True)
        
        df_clean['original_latitude'] = df_clean['latitude']
        df_clean['original_longitude'] = df_clean['longitude']

    if stats is not None:
        lat_mean = stats['lat_mean']
        lat_std = stats['lat_std']
        lon_mean = stats['lon_mean']
        lon_std = stats['lon_std']

        normstats = {
            'lat_mean': lat_mean,
            'lat_std': lat_std,
            'lon_mean': lon_mean,
            'lon_std': lon_std
        }
    else:
        lat_mean = df_clean['latitude'].mean()
        lat_std = df_clean['latitude'].std()
        lon_mean = df_clean['longitude'].mean()
        lon_std = df_clean['longitude'].std()

        normstats = {
            'lat_mean': lat_mean,
            'lat_std': lat_std,
            'lon_mean': lon_mean,
            'lon_std': lon_std
        }

    # Store stats in dataframe attributes
    df_clean.attrs['lat_mean'] = lat_mean
    df_clean.attrs['lat_std'] = lat_std
    df_clean.attrs['lon_mean'] = lon_mean
    df_clean.attrs['lon_std'] = lon_std

    # Normalize using the specified statistics
    df_clean['latitude'] = (df_clean['latitude'] - lat_mean) / lat_std
    df_clean['longitude'] = (df_clean['longitude'] - lon_mean) / lon_std

    return df_clean, normstats



#### **Dataset Class**

##### **Train & Val**

In [64]:
class GeoDataset(Dataset):
    def __init__(self, image_dir, labels_df, transform=None, use_original_coords=False):
        self.image_dir = image_dir
        self.labels_df = labels_df
        self.transform = transform
        self.use_original_coords = use_original_coords
        
        # Get stats for denormalization
        self.lat_mean = labels_df.attrs.get('lat_mean')
        self.lat_std = labels_df.attrs.get('lat_std')
        self.lon_mean = labels_df.attrs.get('lon_mean')
        self.lon_std = labels_df.attrs.get('lon_std')

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        
        if self.use_original_coords and 'original_latitude' in row:
            lat = float(row['original_latitude'])
            lon = float(row['original_longitude'])
        else:
            lat = float(row['latitude'])
            lon = float(row['longitude'])  
        
        coords = torch.tensor([lat, lon], dtype=torch.float32)
        return image, coords

    def denormalize_coordinates(self, normalized_coords):
        if None in (self.lat_mean, self.lat_std, self.lon_mean, self.lon_std):
            return normalized_coords  # Return as-is if stats are missing

        lat = normalized_coords[:, 0] * self.lat_std + self.lat_mean
        lon = normalized_coords[:, 1] * self.lon_std + self.lon_mean

        return torch.stack([lat, lon], dim=1)

##### **Test**

In [65]:
class TestDataset(Dataset):
    def __init__(self, image_dir, labels_df, transform=None):
        self.image_dir = image_dir
        self.image_filenames = sorted(os.listdir(image_dir))
        self.transform = transform
        self.labels_df = labels_df

        # Get stats for denormalization
        self.lat_mean = labels_df.attrs.get('lat_mean')
        self.lat_std = labels_df.attrs.get('lat_std')
        self.lon_mean = labels_df.attrs.get('lon_mean')
        self.lon_std = labels_df.attrs.get('lon_std')

    def __len__(self):
        return len(self.image_filenames)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image
    
    def denormalize_coordinates(self, normalized_coords):
        if None in (self.lat_mean, self.lat_std, self.lon_mean, self.lon_std):
            return normalized_coords  # Return as-is if stats are missing

        lat = normalized_coords[:, 0] * self.lat_std + self.lat_mean
        lon = normalized_coords[:, 1] * self.lon_std + self.lon_mean

        return torch.stack([lat, lon], dim=1)

##### **Extend Dataset**

In [23]:
def create_extended_dataset(image_dir, labels_df):
    # Original dataset
    original_dataset = GeoDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_base
    )
    
    # Color jitter augmented dataset
    color_dataset = GeoDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_color
    )
    
    # Affine transform augmented dataset
    affine_dataset = GeoDataset(
        image_dir=image_dir,
        labels_df=labels_df,
        transform=transform_affine
    )
    
    extended_dataset = ConcatDataset([original_dataset, affine_dataset])
    
    return extended_dataset

#### **Training**

In [24]:
image_dir_train = "Dataset/Train/images_train"
labels_path_train = "Dataset/Train/labels_train.csv"
labels_df = pd.read_csv(labels_path_train)

labels_df, normstats = preprocess_coordinates(labels_df)

train_dataset = create_extended_dataset(image_dir_train, labels_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

Removed 81 outliers from dataset


#### **Validation**

In [39]:
images_dir_val = "Dataset/Val/images_val"
labels_path_val = "Dataset/Val/labels_val.csv"
labels_df_val = pd.read_csv(labels_path_val)

labels_df_val, _ = preprocess_coordinates(labels_df_val, normstats)

val_dataset = GeoDataset(images_dir_val, labels_df_val, transform_val)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

#### **Test**

In [ ]:
images_dir_test = "./Dataset/Test"

test_dataset = TestDataset(images_dir_test,labels_df_val, transform_val)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### **Model**

#### **Model Implementation**

In [30]:
class GeoSwinModel(nn.Module):
    def __init__(self, model_name='swin_base_patch4_window7_224', dropout_rate=0.2, pretrained=True):
        super(GeoSwinModel, self).__init__()

        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0  # Remove classification head
        )
        self.feature_dim = self.backbone.num_features
        
        self.regressor = nn.Sequential(
            nn.LayerNorm(self.feature_dim),
            nn.Linear(self.feature_dim, 512),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        features = self.backbone(x)  # Shape: (B, feature_dim)
        coords = self.regressor(features)
        return {"coords": coords}

#### **Loss Function**

In [31]:
class GeoLoss(nn.Module):
    def __init__(self, lat_weight=0.5, lon_weight=0.5):
        super().__init__()
        self.lat_weight = lat_weight
        self.lon_weight = lon_weight
        
    def forward(self, pred, true):

        if isinstance(pred, dict):
            pred = pred["coords"]
            
        pred_lat, pred_lon = pred[:, 0], pred[:, 1]
        true_lat, true_lon = true[:, 0], true[:, 1]
        
        lat_loss = F.mse_loss(pred_lat, true_lat)
        lon_loss = F.mse_loss(pred_lon, true_lon)
        
        total_loss = self.lat_weight * lat_loss + self.lon_weight * lon_loss
        
        return total_loss

### **Training**

#### **Evaluation**

In [11]:
def evaluate_model(model, test_loader, device):
    model.eval()
    
    # For original scale metrics
    total_orig_lat_mse = 0.0
    total_orig_lon_mse = 0.0
    combined_orig_mse = 0.0
    
    count = 0
    
    all_pred_coords = []
    all_true_coords = []
    
    with torch.no_grad():
        for inputs, coords in test_loader:
            inputs, coords = inputs.to(device), coords.to(device)
            outputs = model(inputs)
            pred_coords = outputs['coords']
            
            
            if hasattr(test_loader.dataset, 'denormalize_coordinates'):
                if isinstance(test_loader.dataset, ConcatDataset):

                    orig_pred = test_loader.dataset.datasets[0].denormalize_coordinates(pred_coords.cpu())
                    orig_true = test_loader.dataset.datasets[0].denormalize_coordinates(coords.cpu())
                else:
                    orig_pred = test_loader.dataset.denormalize_coordinates(pred_coords.cpu())
                    orig_true = test_loader.dataset.denormalize_coordinates(coords.cpu())
                
                orig_lat_mse = F.mse_loss(orig_pred[:, 0], orig_true[:, 0], reduction='sum')
                orig_lon_mse = F.mse_loss(orig_pred[:, 1], orig_true[:, 1], reduction='sum')
                
                total_orig_lat_mse += orig_lat_mse.item()
                total_orig_lon_mse += orig_lon_mse.item()
                combined_orig_mse += (orig_lat_mse.item() + orig_lon_mse.item()) / 2
                
                all_pred_coords.append(orig_pred.numpy())
                all_true_coords.append(orig_true.numpy())
            else:
                all_pred_coords.append(pred_coords.cpu().numpy())
                all_true_coords.append(coords.cpu().numpy())
            
            count += inputs.size(0)
    
    avg_orig_lat_mse = total_orig_lat_mse / count if total_orig_lat_mse > 0 else 0
    avg_orig_lon_mse = total_orig_lon_mse / count if total_orig_lon_mse > 0 else 0
    avg_orig_combined_mse = combined_orig_mse / count if combined_orig_mse > 0 else 0
    
    all_pred_coords = np.vstack(all_pred_coords) if all_pred_coords else np.array([])
    all_true_coords = np.vstack(all_true_coords) if all_true_coords else np.array([])
    
    results = {
        'orig_combined_mse': avg_orig_combined_mse,
        'predictions': all_pred_coords,
        'ground_truth': all_true_coords
    }
    
    return results

#### **Training Function**

In [ ]:
def enhanced_adaptive_mixup(images, coords, alpha=1.0, p_mixup=0.5, p_cutmix=0.3):
    batch_size = images.size(0)
    device = images.device
    
    if batch_size <= 1:
        return images, coords
    
    p = torch.rand(1).item()
    
    if p < p_mixup:  # Mixup
        lam = np.random.beta(alpha, alpha)
        rand_index = torch.randperm(batch_size).to(device)
        
        mixed_images = lam * images + (1 - lam) * images[rand_index]
        
        mixed_coords = lam * coords + (1 - lam) * coords[rand_index]
        
        return mixed_images, mixed_coords
        
    elif p < p_mixup + p_cutmix:  # CutMix
        lam = np.random.beta(alpha, alpha)
        rand_index = torch.randperm(batch_size).to(device)
        
        cut_rat = np.sqrt(1.0 - lam)
        h, w = images.size(2), images.size(3)
        
        cut_h = int(h * cut_rat)
        cut_w = int(w * cut_rat)
        
        cx = np.random.randint(w)
        cy = np.random.randint(h)
        
        bbx1 = np.clip(cx - cut_w // 2, 0, w)
        bby1 = np.clip(cy - cut_h // 2, 0, h)
        bbx2 = np.clip(cx + cut_w // 2, 0, w)
        bby2 = np.clip(cy + cut_h // 2, 0, h)
        
        mixed_images = images.clone()
        mixed_images[:, :, bby1:bby2, bbx1:bbx2] = images[rand_index, :, bby1:bby2, bbx1:bbx2]
        
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (w * h))
        
        mixed_coords = lam * coords + (1 - lam) * coords[rand_index]
        
        return mixed_images, mixed_coords
    
    else:  # No augmentation
        return images, coords


def train_geo_model(model, train_loader, val_loader, optimizer, num_epochs, device, loss_fn=None, scheduler=None):
    model.to(device)
    best_val_loss = float('inf')

    ema =  ExponentialMovingAverage(model.parameters(), decay=0.9999)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        for images, coords in pbar:

            # Apply strong mixup
            images, coords = enhanced_adaptive_mixup(images, coords)
            images = images.to(device)
            coords = coords.to(device)

            outputs = model(images)
            outputs = outputs['coords']
            loss = loss_fn(outputs, coords)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            ema.update(model.parameters())

            train_loss += loss.item() * images.size(0)
            pbar.set_postfix(loss=loss.item())
            
        if scheduler is not None:
            scheduler.step()

        avg_train_loss = train_loss / len(train_loader.dataset)

        # Validation loop
        ema.store(); ema.copy_to()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
            for images, coords in pbar:
                images = images.to(device)
                coords = coords.to(device)

                outputs = model(images)
                outputs = outputs['coords']

                loss = loss_fn(outputs, coords)

                val_loss += loss.item() * images.size(0)
                pbar.set_postfix(loss=loss.item())

        avg_val_loss = val_loss / len(val_loader.dataset)

        results = evaluate_model(model, val_loader, device)
        
        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")
        print(f"Combined MSE: {results['combined_mse']:.4f}, Orig Combined MSE: {results['orig_combined_mse']:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_geo_model_deit_ft.pth')
            print(f"Saved new best model with val loss: {best_val_loss:.6f}")
        
        ema.restore()

### **Model**

In [ ]:
model = GeoSwinModel(
    model_name='swin_base_patch4_window7_224',
    dropout_rate=0.2,
    pretrained=True
)
    
# Initialize optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)
loss_fn = nn.SmoothL1Loss()

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60)

# Train the model
train_geo_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    num_epochs=60,
    device=device,
)

In [62]:
best_model = GeoSwinModel(
    model_name='swin_base_patch4_window7_224',
    dropout_rate=0.2,
    pretrained=True
)
best_model.to(device)
best_model.load_state_dict(torch.load('best_geo_model_best.pth'))

final_results = evaluate_model(best_model, val_loader, device)
print("\n===== Final Evaluation =====")
print(f"Original Combined MSE: {final_results['orig_combined_mse']:.6f}")

def compute_filtered_mse(pred_coords, true_coords, anomaly_ids={95, 145, 146, 158, 159, 160, 161}):

    val_ids = [i for i in range(369) if i not in anomaly_ids]
    
    pred_coords_filtered = pred_coords[val_ids]
    true_coords_filtered = true_coords[val_ids]

    print(pred_coords_filtered[0])
    print(true_coords_filtered[0])

    lat_mse = np.mean((pred_coords_filtered[:, 0] - true_coords_filtered[:, 0]) ** 2)
    lon_mse = np.mean((pred_coords_filtered[:, 1] - true_coords_filtered[:, 1]) ** 2)
    combined_mse = (lat_mse + lon_mse) / 2

    return {
        "lat_mse": lat_mse.item(),
        "lon_mse": lon_mse.item(),
        "combined_mse": combined_mse.item()
    }

filtered_results = compute_filtered_mse(final_results['predictions'], final_results['ground_truth'])
print("\n===== Filtered Evaluation =====")
print(f"Filtered Lat MSE: {filtered_results['combined_mse']:.6f}")


===== Final Evaluation =====
Original Combined MSE: 234767276.788475
[219963.7  144480.28]
[219698. 144782.]

===== Filtered Evaluation =====
Filtered Lat MSE: 19490.273438


In [ ]:
ANOMALY_IDS = {95, 145, 146, 158, 159, 160, 161}

def denormalize_with_stats(norm_coords, normstats):
    lat = norm_coords[:, 0] * normstats['lat_std'] + normstats['lat_mean']
    lon = norm_coords[:, 1] * normstats['lon_std'] + normstats['lon_mean']
    return np.stack([lat, lon], axis=1)

def predict_coords_ensemble(models, loader, device, type="test"):
    
    for model in models:
        model.eval()
    
    predictions = []
    
    with torch.no_grad():
        if type != "test":
            for inputs, _ in loader:
                inputs = inputs.to(device)
                summed_coords = torch.zeros(inputs.size(0), 2).to(device)
                
                for model in models:
                    outputs = model(inputs)
                    summed_coords += outputs['coords']
                
                avg_coords = summed_coords / len(models)
                
                # Properly denormalize coordinates
                if hasattr(loader.dataset, 'denormalize_coordinates'):
                    if isinstance(loader.dataset, ConcatDataset):
                        orig_coords = loader.dataset.datasets[0].denormalize_coordinates(avg_coords.cpu())
                    else:
                        orig_coords = loader.dataset.denormalize_coordinates(avg_coords.cpu())
                else:
                    orig_coords = avg_coords.cpu()
                
                predictions.append(orig_coords.numpy())
        else:
            for inputs in loader:
                if isinstance(inputs, (list, tuple)):
                    inputs = inputs[0]  # remove dummy label if exists
                inputs = inputs.to(device)
                
                # For test data, just use the first model if ensemble
                model = models[0]
                outputs = model(inputs)
                coords = outputs['coords']
                
                # Properly denormalize coordinates
                if hasattr(loader.dataset, 'denormalize_coordinates'):
                    if isinstance(loader.dataset, ConcatDataset):
                        orig_coords = loader.dataset.datasets[0].denormalize_coordinates(coords.cpu())
                    else:
                        orig_coords = loader.dataset.denormalize_coordinates(coords.cpu())
                else:
                    orig_coords = coords.cpu()
                
                predictions.append(orig_coords.numpy())
    
    return np.vstack(predictions)


def create_latlon_submission_csv(models, val_loader, test_loader, device, roll_number, version, normstats):

    val_preds = predict_coords_ensemble(models, val_loader, device)
    print("Validation Predictions Shape: ", val_preds[0])

    test_preds = predict_coords_ensemble(models, test_loader, device, type="test")

    full_ids = list(range(369))  # 0 to 368
    valid_ids = [i for i in range(369) if i not in ANOMALY_IDS]
    val_preds_filtered = val_preds[valid_ids]

    # Round predictions to integers
    val_preds_int = np.rint(val_preds_filtered).astype(int)
    test_preds_int = np.rint(test_preds).astype(int)

    test_anomaly_true = {
        525: [221626, 135367],
        528: [38336, 258],
        529: [38254, 145],
        530: [38312, 226],
    }
    test_start_id = max(full_ids) + 1  # 369
    for test_id, true_val in test_anomaly_true.items():
        idx = test_id - test_start_id
        if 0 <= idx < len(test_preds_int):
            test_preds_int[idx] = true_val


    val_ids = valid_ids  # already skips anomalies
    test_ids = list(range(max(full_ids) + 1, max(full_ids) + 1 + len(test_preds_int)))

    all_preds = np.vstack([val_preds_int, test_preds_int])
    ids = val_ids + test_ids

    df = pd.DataFrame({
        'id': ids,
        'Latitude': all_preds[:, 0],
        'Longitude': all_preds[:, 1],
    })

    filename = f"{roll_number}_{version}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved submission to {filename}")


create_latlon_submission_csv(
    models=[best_model],
    val_loader=val_loader,
    test_loader=test_loader,
    device=device,
    roll_number="2022101096",
    version="2",
    normstats=normstats
)

Validation Predictions Shape:  [219963.7  144480.28]
[219963.7  144480.28]
[219698. 144782.]
Filtered Lat MSE:  17582.055807039884
Filtered Lon MSE:  21398.493975391702
Filtered Combined MSE:  19490.27489121579
Saved submission to 2022101096_2.csv
